In [1]:
from sklearn.model_selection import train_test_split
import json

with open('records-merged.json') as file:
    sessions = json.load(file)

with open('buys2.json') as file:
    buys = json.load(file)

model_units = 64
dense_units = 1

from tensorflow import keras
from tensorflow.keras import layers

In [2]:
import pandas as pd

features = pd.DataFrame(sessions)
labels = pd.DataFrame(buys)

# split to train and test
RANDOM_STATE = 76
TEST_SIZE = 0.2

X_train, X_test, y_train, y_test = train_test_split(features, labels,
                                                    test_size=TEST_SIZE,
                                                    shuffle=False,
                                                    random_state=RANDOM_STATE)
# end test_train_split

# FIT TRANSFORM
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)



In [3]:
attrs_count = X_train.shape[1]

X_train = [X_train]
y_train = [y_train]

In [4]:
model = keras.Sequential()
model.add(layers.Dense(20, activation='relu', input_shape=(1,attrs_count)))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 20)             620       
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              21        
Total params: 641
Trainable params: 641
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.fit(X_train, y_train, batch_size=1, epochs=1, verbose=2)


37165/37165 - 155s - loss: 0.0327


In [6]:
y_pred = model.predict(X_test)
#print(y_pred)
y_pred = [x[0] for x in y_pred]

[0] 0.0020239353
[0] 0.0023138225
[0] 0.0033215582
[0] 0.0034888983
[0] 0.0052785277
[0] 0.0047101974
[0] 0.0021791756
[0] 0.0036772192
[0] 0.003785044
[0] 0.017767757
[0] 0.03162819
[0] 0.0019314885
[0] 0.0045101345
[0] 0.0046078265
[0] 0.002389878
[0] 0.0024357438
[0] 0.0048003793
[0] 0.00224936
[0] 0.0025182664
[0] 0.0027121305
[0] 0.3607061
[0] 0.6526124
[0] 0.0048962235
[0] 0.0038585365
[0] 0.0040944517
[0] 0.0030489266
[0] 0.0041831136
[0] 0.003882438
[0] 0.0028313994
[0] 0.004945487
[0] 0.001734376
[0] 0.002190113
[0] 0.0018246174
[0] 0.0019625127
[0] 0.003538549
[0] 0.0025970042
[0] 0.0027953386
[0] 0.0028742552
[0] 0.0035287142
[0] 0.0023305118
[0] 0.0076707006
[0] 0.083411515
[0] 0.09579542
[0] 0.10215816
[0] 0.101706326
[0] 0.101732016
[0] 0.0926528
[0] 0.1047183
[0] 0.106469214
[0] 0.10590166
[0] 0.0019591153
[0] 1.557377e-05
[0] 0.0046304464
[0] 0.002545029
[0] 0.00044175982
[0] 0.00026026368
[0] 0.0001975894
[0] 0.0001656115
[0] 0.000120459656
[0] 0.0044778585
[0] 0.00608

In [13]:
maxv = 0

for a,b in zip(y_test.values.tolist(), y_pred):
    if a[0] == 0 and b> 0.5:
        print(a,b)

        maxv = max(maxv, b)

print(maxv)
#loss = keras.losses.mean_squared_error(y_test, y_pred)
#print(loss)



[0] 0.6526124
[1] 0.81595325
[1] 0.73548704
[1] 0.54146653
[1] 0.53576696
[1] 0.5787151
[1] 0.5154441
[1] 0.56068474
[0] 0.62925994
[1] 0.51538956
[0] 0.5506837
[1] 0.55087125
[1] 0.70394236
[1] 0.5494705
[0] 0.60296416
[1] 0.55570793
[1] 0.6892442
[1] 0.8158899
[1] 0.7805644
[1] 0.7878473
[1] 0.5073554
[0] 0.75538677
[0] 0.64839596
[0] 0.6680656
[1] 0.6289846
[1] 0.67977166
[0] 0.5064963
[0] 0.77777356
[1] 0.8940557
[0] 0.52146137
[1] 0.69248754
[0] 0.68197644
[0] 0.5380723
[0] 0.796692
[1] 0.62357676
[1] 0.70623064
[1] 0.7610715
[1] 0.54334
[1] 0.77915025
[1] 0.50446725
[1] 0.5149917
[0] 0.5658964
[0] 0.81445146
[1] 0.7471833
[0] 0.7891158
[1] 0.7443171
[0] 0.72089386
[1] 0.7837124
[1] 0.6256433
[1] 0.5144195
[0] 0.64665365
[0] 0.5136513
[1] 0.5361006
[1] 0.7730084
[1] 0.7192747
[1] 0.7109675
[1] 0.50833666
[1] 0.5529068
[0] 0.80687547
[0] 0.6038232
[0] 0.6847893
[1] 0.71142864
[1] 0.7651438
[1] 0.79989564
[1] 0.6621541
[1] 0.568393
[1] 0.7777616
[1] 0.7459193
[0] 0.72184765
[0] 0.75